# Covid vs India
One of the biggest question I have been wondering is , with every adult allowed now to register, Where are vaccines slots available in india ? How can I make this decision programmitcally because trying to find information on government website is rather "sisyphus task".

So with rather self involed goal in mind, We would using publicly avaialbe data try to answer following questions

1. Latest vaccine slots per district and State in India
2. Top 5 States Running behind on schedule
3. Top 5 States Running ahead of everyone else.
4. When can I latest find slots in Delhi, Bombay, Chennai and Bangalore for people between 18-45


- toc: false
- branch: master
- badges: false,
- comments: true,
- categories: [vaccine, covid, jupyter, python],
- image: images/statistics.png,
- hide: false

Thanks to https://github.com/bhattbhavesh91/cowin-vaccination-slot-availability/blob/main/cowin-api-availability.ipynb for doing actual work, I picked up loads of stuff from there.


In [9]:
!{sys.executable} -m pip install --user install requests
!{sys.executable} -m pip install --user install altair
!{sys.executable} -m pip install --user install pandas
!{sys.executable} -m pip install --user install geopandas
#hide

You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/01/55/0bb2226e34f21fa549c3f4557b4f154a5632f61132a969da17c95ca8eab9/altair-4.1.0-py3-none-any.whl
     |████████████████████████████████| 16.0MB 2.0MB/s eta 0:00:01    |████▉                           | 2.4MB 3.0MB/s eta 0:00:05
  Using cached https://files.pythonhosted.org/packages/12/f5/537e55f8ba664ff2a26f26913010fb0fcb98b6bbadc6158af888184fd0b7/toolz-0.11.1-py3-none-any.whl
     |████████████████████████████████| 10.5MB 6.5MB/s eta 0:00:01    |███████████████████████▊        | 7.8MB 2.5MB/s eta 0:00:02
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/d7/bf/e9cefb69d39155d122b6ddca53893b61

  Using cached https://files.pythonhosted.org/packages/cc/ab/85d8da5c9a45e072301beb37ad7f833cd344e04c817d97e0cc75681d248f/munch-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e9/da/824b92d9942f4e472702488857914bdd50f73021efea15b4cad9aca8ecef/click_plugins-1.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/42/1e/947eadf10d6804bf276eb8a038bd5307996dceaaa41cfd21b7a15ec62f5d/cligj-0.7.1-py3-none-any.whl
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.


Get All libraries in place

In [59]:
import geopandas as gpd
import pandas as pd
import requests
import json
from collections import defaultdict
from dataclasses import dataclass, asdict
import datetime
from typing import List

Lets make a data class to store our geographical and vaccination data

In [52]:
@dataclass
class District:
    id:str = None
    name:str = None
    state_id:int = None


@dataclass
class Session:
    id:str
    date:str
    available_capacity:int
    min_age_limit:int
    vaccine:str
    center_id:str
    district_id:str
        
@dataclass
class Center:
    id:str
    name:str
    state_name:str
    district_name:str
    block_name:str
    lat:int
    lng:int
    fee_type:str
    session_date: List[Session]
    district_id:str


Lets call the API to get the populate geographical data

In [64]:
MOZILLA_HEADER = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
GET_DISTRICT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}"
STATE_TO_DISTRICT_MAPPING = defaultdict(list)
DISTRICTS = defaultdict(District)
DISTRICT_VACCINATION_DATA = defaultdict(list)

STATE_ID_TO_STATE_NAME_MAPPING = {
    1:"Andaman and Nicobar",
    2:"",
    3:"",
    4:"",
    5:"",
    6:"",
    7:"",
    8:"",
    9:"",
    10:"",
    11:"",
    12:"",
    13:"",
    14:"",
    15:"",
    16:"",
    17:"",
    18:"",
    19:"",
    20:"",
    21:"",
    22:"",
    23:"",
    24:"",
    25:"",
    26:"",
    27:"",
    28:"",
    29:"",
    30:"",
    31:"",
    32:"",
    33:"",
    34:"",
    35:"",
    36:"",
    37:"",
    38:"",
    39:"",
    40:"",
}

for state_code in STATE_ID_TO_STATE_NAME_MAPPING.keys():
    headers = {'User-Agent': MOZILLA_HEADER}
    response = requests.get(GET_DISTRICT_DATA_API_URL.format(state_code), headers=headers)
    districts_data = json.loads(response.content)
    for district in districts_data['districts']:
        district_name = district['district_name']
        district_id = district['district_id']
        district = District(name = district_name, id = district_id , state_id=state_code) 
        STATE_TO_DISTRICT_MAPPING[state_code].append(district)
        DISTRICTS[district_id] = district

defaultdict(<class 'list'>, {1: [District(id=3, name='Nicobar', state_id=1), District(id=1, name='North and Middle Andaman', state_id=1), District(id=2, name='South Andaman', state_id=1)], 2: [District(id=9, name='Anantapur', state_id=2), District(id=10, name='Chittoor', state_id=2), District(id=11, name='East Godavari', state_id=2), District(id=5, name='Guntur', state_id=2), District(id=4, name='Krishna', state_id=2), District(id=7, name='Kurnool', state_id=2), District(id=12, name='Prakasam', state_id=2), District(id=13, name='Sri Potti Sriramulu Nellore', state_id=2), District(id=14, name='Srikakulam', state_id=2), District(id=8, name='Visakhapatnam', state_id=2), District(id=15, name='Vizianagaram', state_id=2), District(id=16, name='West Godavari', state_id=2), District(id=6, name='YSR District, Kadapa (Cuddapah)', state_id=2)], 3: [District(id=22, name='Anjaw', state_id=3), District(id=20, name='Changlang', state_id=3), District(id=25, name='Dibang Valley', state_id=3), District(

In [63]:
DISTRICT_DF = pd.DataFrame.from_dict([asdict(district) for district in DISTRICTS.values()])
STATE_TO_DISTRICT_MAPPING_DF = pd.DataFrame.from_dict()
print(DISTRICT_DF)

      id                      name  state_id
0      3                   Nicobar         1
1      1  North and Middle Andaman         1
2      2             South Andaman         1
3      9                 Anantapur         2
4     10                  Chittoor         2
..   ...                       ...       ...
756  791                    Chamba        39
757  795                    Kangra        39
758  792                   Kinnaur        39
759  793                     Mandi        39
760  794                    Shimla        39

[761 rows x 3 columns]


Lets now call actual api to get slots 

In [28]:
MAX_DAYS = 15
DIST_ID = 367
def get_data_fiften_days_in_future_from_today():
    base = datetime.datetime.today()
    date_list = [base + datetime.timedelta(days=x) for x in range(MAX_DAYS)]
    return [x.strftime("%d-%m-%Y") for x in date_list]

for INP_DATE in get_data_fiften_days_in_future_from_today():
    URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(DIST_ID, INP_DATE)
    response = requests.get(URL)
    if response.ok:
        resp_json = response.json()
        if resp_json["centers"]:
            print("You can book vaccine slots on {}".format(INP_DATE))
        else:
            print("No available slots on {}".format(INP_DATE))

b'{"centers":[{"center_id":571829,"name":"6. SultanpurPHC Session","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Lonar","pincode":443302,"lat":19,"long":76,"from":"09:00:00","to":"17:00:00","fee_type":"Free","sessions":[{"session_id":"e072a41a-b407-47bf-91ae-6fb576b7035c","date":"30-04-2021","available_capacity":50,"min_age_limit":45,"vaccine":"","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"]},{"session_id":"6f10eeb6-ec96-4a23-91eb-a3cf0a08eb22","date":"01-05-2021","available_capacity":44,"min_age_limit":45,"vaccine":"","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"]}]},{"center_id":582816,"name":"13. Sindkhed Raja RH","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Sindkhed Raja","pincode":443203,"lat":19,"long":76,"from":"09:00:00","to":"17:00:00","fee_type":"Free","sessions":[{"session_id":"1128697e-6aff-43a4-bcee-f63610b49f83","date":"30-04-2021","available_capacity":29,"m

b'{"centers":[{"center_id":563393,"name":"Deolgaon Raja RH Campus","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Deolgaon Raja","pincode":443204,"lat":20,"long":76,"from":"09:00:00","to":"17:00:00","fee_type":"Free","sessions":[{"session_id":"2d38d2c1-828a-4125-b113-0283cd8b60a1","date":"02-05-2021","available_capacity":35,"min_age_limit":45,"vaccine":"","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"]}]},{"center_id":150792,"name":"Raigaon PHC","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Lonar","pincode":443302,"lat":0,"long":0,"from":"09:00:00","to":"17:00:00","fee_type":"Free","sessions":[{"session_id":"61cc9bbc-d916-4055-ab46-d1cffafbb4c9","date":"02-05-2021","available_capacity":49,"min_age_limit":45,"vaccine":"","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"]},{"session_id":"f8f16fdd-2631-4632-8643-3d4a5ad18ab4","date":"03-05-2021","available_capacity":49,"min_age_limi

b'{"centers":[{"center_id":613551,"name":"Dhanwe Hospital Chikhali","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Chikhli","pincode":443201,"lat":20,"long":76,"from":"10:00:00","to":"17:00:00","fee_type":"Paid","sessions":[{"session_id":"d25ddaba-a726-4c08-9d95-ec11938dae95","date":"04-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["10:00AM-12:00PM","12:00PM-02:00PM","02:00PM-04:00PM","04:00PM-05:00PM"]},{"session_id":"c0b96619-8b4a-4dce-948d-f40dcfae5663","date":"05-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["10:00AM-12:00PM","12:00PM-02:00PM","02:00PM-04:00PM","04:00PM-05:00PM"]}],"vaccine_fees":[{"vaccine":"COVISHIELD","fee":"0"}]},{"center_id":151657,"name":"Warwat Bakal RH","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Sangrampur","pincode":444202,"lat":21,"long":76,"from":"09:00:00","to":"17:00:00","fee_type":"Free","sessions":[{"session_id":"a2da165d-d6a9-4b20-

b'{"centers":[{"center_id":606952,"name":"Kothari Hospital Chikhli","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Chikhli","pincode":443201,"lat":20,"long":76,"from":"10:00:00","to":"17:00:00","fee_type":"Paid","sessions":[{"session_id":"5dab309a-053e-477f-a83f-bd68c9667ca9","date":"07-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["10:00AM-12:00PM","12:00PM-02:00PM","02:00PM-04:00PM","04:00PM-05:00PM"]},{"session_id":"6076846e-033c-47f0-ad8f-d9d434a2090c","date":"08-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-06:00PM"]},{"session_id":"afd2c1fb-7bad-4772-a642-fd7516c21ac0","date":"09-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-06:00PM"]},{"session_id":"9d2613c6-9b0d-49bb-9b30-23fe09381980","date":"10-05-2021","available_capacity

b'{"centers":[{"center_id":149816,"name":"Kolwad SC","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Buldhana","pincode":443106,"lat":0,"long":0,"from":"09:00:00","to":"18:00:00","fee_type":"Free","sessions":[{"session_id":"c1810750-e9ea-4f8e-9c21-098b6e782b54","date":"09-05-2021","available_capacity":50,"min_age_limit":45,"vaccine":"","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-06:00PM"]},{"session_id":"fdae6e51-8d03-4e12-b63b-dc7d7c6483c0","date":"10-05-2021","available_capacity":49,"min_age_limit":45,"vaccine":"","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-06:00PM"]},{"session_id":"3ba5d03d-aaed-4e8d-9714-ff7092668e1b","date":"11-05-2021","available_capacity":50,"min_age_limit":45,"vaccine":"","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-06:00PM"]}]},{"center_id":606952,"name":"Kothari Hospital Chikhli","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Chikhli","p

b'{"centers":[{"center_id":604816,"name":"MEHETRE HOSPITAL","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Buldhana","pincode":443001,"lat":20,"long":76,"from":"10:00:00","to":"17:00:00","fee_type":"Paid","sessions":[{"session_id":"b6c95153-d3e1-4c28-85d4-4ae99f1fe814","date":"11-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["10:00AM-12:00PM","12:00PM-02:00PM","02:00PM-04:00PM","04:00PM-05:00PM"]},{"session_id":"e1ba59e2-f904-4948-afa1-0a877e98f769","date":"12-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["10:00AM-12:00PM","12:00PM-02:00PM","02:00PM-04:00PM","04:00PM-05:00PM"]},{"session_id":"1a5d5e30-bba8-4fed-b4b1-6c5c372e4605","date":"13-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["10:00AM-12:00PM","12:00PM-02:00PM","02:00PM-04:00PM","04:00PM-05:00PM"]},{"session_id":"703dddc5-aa9a-4a1f-83fc-818978adc3bd","date":"14-05-2021","available_capacity":0,"mi

b'{"centers":[{"center_id":604816,"name":"MEHETRE HOSPITAL","state_name":"Maharashtra","district_name":"Buldhana","block_name":"Buldhana","pincode":443001,"lat":20,"long":76,"from":"10:00:00","to":"17:00:00","fee_type":"Paid","sessions":[{"session_id":"703dddc5-aa9a-4a1f-83fc-818978adc3bd","date":"14-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["10:00AM-12:00PM","12:00PM-02:00PM","02:00PM-04:00PM","04:00PM-05:00PM"]},{"session_id":"2d25f26a-6d27-4019-8611-c6d9f1b9e1f4","date":"15-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["10:00AM-12:00PM","12:00PM-02:00PM","02:00PM-04:00PM","04:00PM-05:00PM"]},{"session_id":"0fc5164e-73d0-4490-a96f-1207bc02bce0","date":"16-05-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["10:00AM-12:00PM","12:00PM-02:00PM","02:00PM-04:00PM","04:00PM-05:00PM"]},{"session_id":"a8ce751d-62d4-4bee-854d-916648f41ef7","date":"17-05-2021","available_capacity":0,"mi